# Grid of Resistors I
## Compute the effective resistance of a 2n+1 by 2n+2 grid of resistors
## Using SOR with red-black ordering

The problem is to compute the voltages and the effective resistance of a 2n+1 by 2n+2 grid of 1 ohm resistors if a battery is connected to the two center points. This is a discrete version of finding the lines of force using iron filings for a magnet.

The pictures below describe the two dimensional problem. We will use this problem as a common theme when introducing various aspects of Julia ranging from optimizing sequential code to GPU and distributed computing in Julia.

<img src='battery.gif'>

The method of solution that we will use here is successive overrelaxation (SOR) with red-black ordering. This is certainly not the fastest way to solve the problem, but it does illustrate many important programming ideas. 

It is not so important that you know the details of SOR. (Some of the basic ideas may be found on pages 407-409 of Gil Strang's Introduction to Applied Mathematics, a somewhat more in depth discussion may be found in any serious numerical analysis text such as Stoer and Bulirsch's Introduction to Numerical Analysis.) What is important is that you see that the nodes are divided in half into red nodes and black nodes. During the first pass, the red nodes obtain the voltages as a weighted average of their original voltage, the input (if any) and the four surrounding black nodes. During the second pass, the black nodes obtain voltages from the four surrounding red nodes. The process converges in the limit to the correct answer for the finite grid. The following Julia code solves the 2D problem: 

In [ ]:
# First load some package not directly related to the resistors problem
using BenchmarkTools
using PlotlyJS
using Interact

function compute_resistance(n, nreps = 100)
    # Original MATLAB version, Alan Edelman, January 1994
    # Julia translations, Andreas Noack, June 2018

    # assume n and omega already defined or take
    # the following values for the optimal omega
    μ = (cos(π/(2*n)) + cos(π/(2*n + 1)))/2
    ω = 2*(1 - sqrt(1 - μ^2))/μ^2
    # (See page 409 of Strang Intro to Applied Math , this is equation 16)

    # Initialize voltages
    v = zeros(Float32, 2*n + 1, 2*n + 2)

    # Define Input Currents
    b = copy(v)
    b[n + 1, (n + 1):(n + 2)]  = [1 -1]

    # Makes indices easy to read
    ie = 2:2:(2*n)      # even i's
    io = 3:2:(2*n - 1)  # odd i's
    je = 2:2:(2*n)      # even j's
    jo = 3:2:(2*n + 1)  # odd j's

    # Jacobi Steps
    for k in 1:nreps
        v[ie, je] = (1 - ω) * v[ie,je] +
                      ω*(v[ie + 1, je] + v[ie - 1, je] + v[ie, je + 1] + v[ie, je - 1] + b[ie, je])/4
        v[io, jo] = (1 - ω) * v[io, jo] +
                      ω*(v[io + 1, jo] + v[io - 1, jo] + v[io, jo + 1] + v[io, jo - 1] + b[io, jo])/4
        v[ie, jo] = (1 - ω) * v[ie, jo] +
                      ω*(v[ie + 1, jo] + v[ie - 1, jo] + v[ie, jo + 1] + v[ie, jo - 1] + b[ie, jo])/4
        v[io, je] = (1 - ω) * v[io, je] +
                      ω*(v[io + 1, je] + v[io - 1, je] + v[io, je + 1] + v[io, je - 1] + b[io, je])/4
    end
    # Compute resistance = v_A - v_b = 2 v_A
    r = 2*v[n + 1, n + 1]
    return v, r
end

#               n=3: 81466/167063
#               n=4: 0.49279782192042 

A contour plot of the answer as a function of $n$ looks like

In [ ]:
using Interact
@manipulate for i in 1:10
    v, r = compute_resistance(4, i)
    plot(contour(z = v), Layout(yaxis_scaleanchor="x"))
end

In this notebook, we will look at the performance of the code, how to make it faster with "broadcsting" and to get familiar with various ways to inspect the code to get an understanding of what is going on.

First, we'll use the `@btime` macro to time the function and see how much memory it allocated.

In [ ]:
@btime compute_resistance(400);

The timings reveal that the function is allocating a lot. Notice that this number is the accumuated number of allocations so most of the memory is already freed again by Julia's garbage collector. However, garbage collection halts the execution of the program and allocating memory from the system takes time, so a first optimization of Julia code is usually to reduce the allocations.

**Exercise**: Look at the source code for `compute_resistance` and try to identify the places where memory might be allocated.

Now we will utilize dot-broadcasting to reduce the number of temporary arrays created. See more at https://julialang.org/blog/2017/01/moredots (the implementation has changed a bit but the semantics are mostly the same).

Broadcasting in Julia serves several. First of all, it allows for convenient operations such as

In [ ]:
zeros(Int, 4) .+ 1

and

In [ ]:
(0:10).+(0:10)'

but it provides an easy way to reduce the number of allocations by *fusing*. To allustrate this, consider the two functions

In [ ]:
f1(x::Vector) = x + x + x
f2(x::Vector) = x .+ x .+ x

**Exercise**: Allocate a random vector of length 10 and use `@btime` to compare the two functions `f1` and `f2`, e.g. `@btime f1($x)`. Explain the difference. (The `$` is a timing technicality that ensures that the Julia compiler doesn't confuse the a global variable `x` and a local variable `x`.)

**Exercise**: Try prefixing a call to `f1` and `f2` with `@code_lowered` to see how the Julia parser reads the two functions differently. The output might be hard to understand but try to read it and interpret the difference.

**Exercise**: Now try to utilize broadcasting to write `compute_resistance_bc` that will allocate less than the original version. Use `@btime` to time and compare to the original.

**Hint**: In the final version, the number of allocations should be roughly `(9607 allocations: 2.87 GiB)` for `n=400`.